<a href="https://colab.research.google.com/github/matbot/LinearProgrammingPython/blob/master/MinimizingCost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@hw6p1
!pip install pulp
from pulp import *

from prettytable import PrettyTable

In [0]:
# Instantiate the problem
model = LpProblem("Minimize Shipping-Transshipment Costs", LpMinimize)

# Construct decision variables
plants = ['p1', 'p2', 'p3', 'p4']
warehouses = ['w1', 'w2', 'w3']
retailers = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7']
pw = LpVariable.dicts("Plant to Warehouse Cost",
                     ((i, j) for i in plants for j in warehouses),
                      lowBound=0,
                      cat='Integer')
wr = LpVariable.dicts("Warehouse to Retailer Cost",
                     ((i, j) for i in warehouses for j in retailers),
                      lowBound=0,
                      cat='Integer')

# Objective Function
model += (10*pw['p1', 'w1'] + 15*pw['p1','w2'] + 11*pw['p2','w1'] + 
         8*pw['p2','w2'] + 13*pw['p3','w1'] + 8*pw['p3','w2'] + 
         9*pw['p3','w3'] + 14*pw['p4','w2'] + 8*pw['p4','w3'] + 
         5*wr['w1','r1'] + 6*wr['w1','r2'] + 7*wr['w1','r3'] + 
         10*wr['w1','r4'] + 12*wr['w2','r3'] + 8*wr['w2','r4'] + 
         10*wr['w2','r5'] + 14*wr['w2','r6'] + 14*wr['w3','r4'] + 
         12*wr['w3','r5'] + 12*wr['w3','r6'] + 6*wr['w3','r7'])

# Constraints
# Supply Constraints
model += pw['p1','w1'] + pw['p1','w2'] == 150
model += pw['p2','w1'] + pw['p2','w2'] == 450
model += pw['p3','w1'] + pw['p3','w2'] + pw['p3','w3'] == 250
model += pw['p4','w2'] + pw['p4','w3'] == 150

# Demand Constraints
model += wr['w1','r1'] == 100
model += wr['w1','r2'] == 150
model += wr['w1','r3'] + wr['w2','r3'] == 100
model += wr['w1','r4'] + wr['w2','r4'] + wr['w3','r4'] == 200
model += wr['w2','r5'] + wr['w3','r5'] == 200
model += wr['w2','r6'] + wr['w3','r6'] == 150
model += wr['w3','r7'] == 100

# Transshipment Constraints
model += lpSum([pw[i, 'w1'] for i in plants]) - lpSum([wr['w1', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w2'] for i in plants]) - lpSum([wr['w2', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w3'] for i in plants]) - lpSum([wr['w3', j] for j in retailers]) == 0

# Block non shippable routes
model += pw['p1','w3'] == 0
model += pw['p2','w3'] == 0
model += pw['p4','w1'] == 0
model += wr['w1','r5'] == 0
model += wr['w1','r6'] == 0
model += wr['w1','r7'] == 0
model += wr['w2','r1'] == 0
model += wr['w2','r2'] == 0
model += wr['w2','r7'] == 0
model += wr['w3','r1'] == 0
model += wr['w3','r2'] == 0
model += wr['w3','r3'] == 0

# Solve it
model.solve()

# Output results
print("Transshipment Model Optimization Status: {}".format(LpStatus[model.status]))
if model.status == 1:
  # If the model is optimal, show the shipping tables.
  print("Optimized Minimum Shipping Cost = ${}".format(round(pulp.value(model.objective),2)))
  
  # Build plant to warehouse table
  print("\nPlant to Warehouse Optimal Shipping Schedule")
  x = PrettyTable()
  x.field_names = ["Plant", "Warehouse", "Units to Ship"]
  for i in pw:
    if pw[i].varValue != 0:
      x.add_row([i[0], i[1], int(pw[i].varValue)])
  print(x)
  
  # Build warehouse to retailer table
  print("\nWarehouse to Retailer Optimal Shipping Schedule")
  y = PrettyTable()
  y.field_names = ["Warehouse", "Retailer", "Units to Ship"]
  for i in wr:
    if wr[i].varValue != 0:
      y.add_row([i[0], i[1], int(wr[i].varValue)])
  print(y)

Transshipment Model Optimization Status: Optimal
Optimized Minimum Shipping Cost = $17100.0

Plant to Warehouse Optimal Shipping Schedule
+-------+-----------+---------------+
| Plant | Warehouse | Units to Ship |
+-------+-----------+---------------+
|   p1  |     w1    |      150      |
|   p2  |     w1    |      200      |
|   p2  |     w2    |      250      |
|   p3  |     w2    |      150      |
|   p3  |     w3    |      100      |
|   p4  |     w3    |      150      |
+-------+-----------+---------------+

Warehouse to Retailer Optimal Shipping Schedule
+-----------+----------+---------------+
| Warehouse | Retailer | Units to Ship |
+-----------+----------+---------------+
|     w1    |    r1    |      100      |
|     w1    |    r2    |      150      |
|     w1    |    r3    |      100      |
|     w2    |    r4    |      200      |
|     w2    |    r5    |      200      |
|     w3    |    r6    |      150      |
|     w3    |    r7    |      100      |
+-----------+----------

In [0]:
# Instantiate the problem
model = LpProblem("Minimize Shipping-Transshipment Costs", LpMinimize)

# Construct decision variables
plants = ['p1', 'p2', 'p3', 'p4']
warehouses = ['w1', 'w2', 'w3']
retailers = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7']
pw = LpVariable.dicts("Plant to Warehouse Cost",
                     ((i, j) for i in plants for j in warehouses),
                      lowBound=0,
                      cat='Integer')
wr = LpVariable.dicts("Warehouse to Retailer Cost",
                     ((i, j) for i in warehouses for j in retailers),
                      lowBound=0,
                      cat='Integer')

# Objective Function
model += (10*pw['p1', 'w1'] + 15*pw['p1','w2'] + 11*pw['p2','w1'] + 
         8*pw['p2','w2'] + 13*pw['p3','w1'] + 8*pw['p3','w2'] + 
         9*pw['p3','w3'] + 14*pw['p4','w2'] + 8*pw['p4','w3'] + 
         5*wr['w1','r1'] + 6*wr['w1','r2'] + 7*wr['w1','r3'] + 
         10*wr['w1','r4'] + 12*wr['w2','r3'] + 8*wr['w2','r4'] + 
         10*wr['w2','r5'] + 14*wr['w2','r6'] + 14*wr['w3','r4'] + 
         12*wr['w3','r5'] + 12*wr['w3','r6'] + 6*wr['w3','r7'])

# Constraints
# Supply Constraints
model += pw['p1','w1'] + pw['p1','w2'] == 150
model += pw['p2','w1'] + pw['p2','w2'] == 450
model += pw['p3','w1'] + pw['p3','w2'] + pw['p3','w3'] == 250
model += pw['p4','w2'] + pw['p4','w3'] == 150

# Demand Constraints
model += wr['w1','r1'] == 100
model += wr['w1','r2'] == 150
model += wr['w1','r3'] + wr['w2','r3'] == 100
model += wr['w1','r4'] + wr['w2','r4'] + wr['w3','r4'] == 200
model += wr['w2','r5'] + wr['w3','r5'] == 200
model += wr['w2','r6'] + wr['w3','r6'] == 150
model += wr['w3','r7'] == 100

# Transshipment Constraints
model += lpSum([pw[i, 'w1'] for i in plants]) - lpSum([wr['w1', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w2'] for i in plants]) - lpSum([wr['w2', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w3'] for i in plants]) - lpSum([wr['w3', j] for j in retailers]) == 0

# Block non shippable routes
model += pw['p1','w3'] == 0
model += pw['p2','w3'] == 0
model += pw['p4','w1'] == 0
model += wr['w1','r5'] == 0
model += wr['w1','r6'] == 0
model += wr['w1','r7'] == 0
model += wr['w2','r1'] == 0
model += wr['w2','r2'] == 0
model += wr['w2','r7'] == 0
model += wr['w3','r1'] == 0
model += wr['w3','r2'] == 0
model += wr['w3','r3'] == 0

# Close warehouse 2
model += lpSum([pw[i, 'w2'] for i in plants]) == 0
model += lpSum([wr['w2', i] for i in retailers]) == 0

# Solve it
model.solve()

# Output results
print("Transshipment Model Optimization Status: {}".format(LpStatus[model.status]))
if model.status == 1:
  # If the model is optimal, show the shipping tables.
  print("Optimized Minimum Shipping Cost = ${}".format(round(pulp.value(model.objective),2)))
  
  # Build plant to warehouse table
  print("\nPlant to Warehouse Optimal Shipping Schedule")
  x = PrettyTable()
  x.field_names = ["Plant", "Warehouse", "Units to Ship"]
  for i in pw:
    if pw[i].varValue != 0:
      x.add_row([i[0], i[1], int(pw[i].varValue)])
  print(x)
  
  # Build warehouse to retailer table
  print("\nWarehouse to Retailer Optimal Shipping Schedule")
  y = PrettyTable()
  y.field_names = ["Warehouse", "Retailer", "Units to Ship"]
  for i in wr:
    if wr[i].varValue != 0:
      y.add_row([i[0], i[1], int(wr[i].varValue)])
  print(y)

Transshipment Model Optimization Status: Infeasible


In [0]:
# Instantiate the problem
model = LpProblem("Minimize Shipping-Transshipment Costs", LpMinimize)

# Construct decision variables
plants = ['p1', 'p2', 'p3', 'p4']
warehouses = ['w1', 'w2', 'w3']
retailers = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7']
pw = LpVariable.dicts("Plant to Warehouse Cost",
                     ((i, j) for i in plants for j in warehouses),
                      lowBound=0,
                      cat='Integer')
wr = LpVariable.dicts("Warehouse to Retailer Cost",
                     ((i, j) for i in warehouses for j in retailers),
                      lowBound=0,
                      cat='Integer')

# Objective Function
model += (10*pw['p1', 'w1'] + 15*pw['p1','w2'] + 11*pw['p2','w1'] + 
         8*pw['p2','w2'] + 13*pw['p3','w1'] + 8*pw['p3','w2'] + 
         9*pw['p3','w3'] + 14*pw['p4','w2'] + 8*pw['p4','w3'] + 
         5*wr['w1','r1'] + 6*wr['w1','r2'] + 7*wr['w1','r3'] + 
         10*wr['w1','r4'] + 12*wr['w2','r3'] + 8*wr['w2','r4'] + 
         10*wr['w2','r5'] + 14*wr['w2','r6'] + 14*wr['w3','r4'] + 
         12*wr['w3','r5'] + 12*wr['w3','r6'] + 6*wr['w3','r7'])

# Constraints
# Supply Constraints
model += pw['p1','w1'] + pw['p1','w2'] == 150
model += pw['p2','w1'] + pw['p2','w2'] == 450
model += pw['p3','w1'] + pw['p3','w2'] + pw['p3','w3'] == 250
model += pw['p4','w2'] + pw['p4','w3'] == 150

# Demand Constraints
model += wr['w1','r1'] == 100
model += wr['w1','r2'] == 150
model += wr['w1','r3'] + wr['w2','r3'] == 100
model += wr['w1','r4'] + wr['w2','r4'] + wr['w3','r4'] == 200
model += wr['w2','r5'] + wr['w3','r5'] == 200
model += wr['w2','r6'] + wr['w3','r6'] == 150
model += wr['w3','r7'] == 100

# Transshipment Constraints
model += lpSum([pw[i, 'w1'] for i in plants]) - lpSum([wr['w1', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w2'] for i in plants]) - lpSum([wr['w2', j] for j in retailers]) == 0
model += lpSum([pw[i, 'w3'] for i in plants]) - lpSum([wr['w3', j] for j in retailers]) == 0

# Block non shippable routes
model += pw['p1','w3'] == 0
model += pw['p2','w3'] == 0
model += pw['p4','w1'] == 0
model += wr['w1','r5'] == 0
model += wr['w1','r6'] == 0
model += wr['w1','r7'] == 0
model += wr['w2','r1'] == 0
model += wr['w2','r2'] == 0
model += wr['w2','r7'] == 0
model += wr['w3','r1'] == 0
model += wr['w3','r2'] == 0
model += wr['w3','r3'] == 0

# Reduce Warehouse 2 shipping capacity
model += lpSum([pw[i, 'w2'] for i in plants]) <= 100

# Solve it
model.solve()

# Output results
print("Transshipment Model Optimization Status: {}".format(LpStatus[model.status]))
if model.status == 1:
  # If the model is optimal, show the shipping tables.
  print("Optimized Minimum Shipping Cost = ${}".format(round(pulp.value(model.objective),2)))
  
  # Build plant to warehouse table
  print("\nPlant to Warehouse Optimal Shipping Schedule")
  x = PrettyTable()
  x.field_names = ["Plant", "Warehouse", "Units to Ship"]
  for i in pw:
    if pw[i].varValue != 0:
      x.add_row([i[0], i[1], int(pw[i].varValue)])
  print(x)
  
  # Build warehouse to retailer table
  print("\nWarehouse to Retailer Optimal Shipping Schedule")
  y = PrettyTable()
  y.field_names = ["Warehouse", "Retailer", "Units to Ship"]
  for i in wr:
    if wr[i].varValue != 0:
      y.add_row([i[0], i[1], int(wr[i].varValue)])
  print(y)

Transshipment Model Optimization Status: Optimal
Optimized Minimum Shipping Cost = $18300.0

Plant to Warehouse Optimal Shipping Schedule
+-------+-----------+---------------+
| Plant | Warehouse | Units to Ship |
+-------+-----------+---------------+
|   p1  |     w1    |      150      |
|   p2  |     w1    |      350      |
|   p2  |     w2    |      100      |
|   p3  |     w3    |      250      |
|   p4  |     w3    |      150      |
+-------+-----------+---------------+

Warehouse to Retailer Optimal Shipping Schedule
+-----------+----------+---------------+
| Warehouse | Retailer | Units to Ship |
+-----------+----------+---------------+
|     w1    |    r1    |      100      |
|     w1    |    r2    |      150      |
|     w1    |    r3    |      100      |
|     w1    |    r4    |      150      |
|     w2    |    r4    |       50      |
|     w2    |    r5    |       50      |
|     w3    |    r5    |      150      |
|     w3    |    r6    |      150      |
|     w3    |    r7 